# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Attempt-to-Fix-SSL-Issue" data-toc-modified-id="Attempt-to-Fix-SSL-Issue-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Attempt to Fix SSL Issue</a></div><div class="lev1 toc-item"><a href="#Part-1---Scraping-The-Federalist-Papers" data-toc-modified-id="Part-1---Scraping-The-Federalist-Papers-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Part 1 - Scraping The Federalist Papers</a></div>

# Attempt to Fix SSL Issue

Received the following error when trying to scrap the federalist papers using the Python 3.8 conda environment:

```
SSLError: HTTPSConnectionPool(host='avalon.law.yale.edu', port=443): Max retries exceeded with url: /18th_century/fed01.asp (Caused by SSLError("Can't connect to HTTPS URL because the SSL module is not available."))
```

Possibly [this SO answer](https://stackoverflow.com/a/45909353) is the reason why. I tried [this SO answer](https://stackoverflow.com/a/55632553) by adding to the shell `PATH` variable (did not modify system `PATH`), but no dice. I also tried `verify=False` in the `get` method but still have the same error.

In [1]:
# Shell command to find out where anaconda is installed
anaconda_exec = !where anaconda
anaconda_exec

['C:\\Users\\Sonya\\Anaconda3\\Scripts\\anaconda.exe']

In [7]:
# Modify path 
import sys
sys.path.append("C:\\Users\\Sonya\\Anaconda3")
sys.path.append("C:\\Users\\Sonya\\Anaconda3\\Scripts")
sys.path.append("C:\\Users\\Sonya\\Anaconda3\\Library\\bin")
sys.path.append("C:\\Users\\Sonya\\Anaconda3\\envs\\scipybase_Jun2021")
sys.path.append("C:\\Users\\Sonya\\Anaconda3\\envs\\scipybase_Jun2021\\Scripts")
sys.path.append("C:\\Users\\Sonya\\Anaconda3\\envs\\scipybase_Jun2021\\Library\\bin")

In [8]:
# Verify path env variable
sys.path

['C:\\Users\\Sonya\\Box Sync\\Projects\\workshop-yale-som-python\\notebooks',
 'C:\\Users\\Sonya\\Box Sync\\Projects\\python-wrappers-and-snippets',
 'C:\\Users\\Sonya\\Box Sync\\Projects\\consulting-data-science\\170220_ChazBatt_SecuritiesTransactions\\deliverables\\securitytrx',
 'C:\\Users\\Sonya\\Box Sync\\Projects\\consulting-data-science\\181106_AdityaSavara_PyMC\\bivariate_ea_k0\\0ModelInputs',
 'C:\\Users\\Sonya\\Box Sync\\Projects\\workshop-yale-som-python\\notebooks',
 'C:\\Users\\Sonya\\Anaconda3\\envs\\scipybase_Jun2021\\python38.zip',
 'C:\\Users\\Sonya\\Anaconda3\\envs\\scipybase_Jun2021\\DLLs',
 'C:\\Users\\Sonya\\Anaconda3\\envs\\scipybase_Jun2021\\lib',
 'C:\\Users\\Sonya\\Anaconda3\\envs\\scipybase_Jun2021',
 '',
 'C:\\Users\\Sonya\\Anaconda3\\envs\\scipybase_Jun2021\\lib\\site-packages',
 'C:\\Users\\Sonya\\Anaconda3\\envs\\scipybase_Jun2021\\lib\\site-packages\\win32',
 'C:\\Users\\Sonya\\Anaconda3\\envs\\scipybase_Jun2021\\lib\\site-packages\\win32\\lib',
 'C:\\Use

# Part 1 - Scraping The Federalist Papers ###

In [9]:
import requests
from bs4 import BeautifulSoup
import os
from pathlib import Path  # This is a useful class for constructing file system paths
import time

In [10]:
# Set the data folder and the number of papers to scrape (papers are identified by integers)
datapath = Path(r"../data/federalist_papers")
# num_papers = 85
num_papers = 2

In [11]:
# Retrieve the contents of all the federalist papers from the law school website and save to our data folder
for i in range(num_papers):
    print(i+1)
    if i < 9:
        ID = "0" + str(i+1)
    else:
        ID = str(i+1)
        
    # Request HTML of the page for the i^th paper
    r = requests.get("http://avalon.law.yale.edu/18th_century/fed%s.asp" % ID, verify=False)
    s = BeautifulSoup(r.text)  # this is the HTML contents of the returned request
    ps = s.findAll("p")  # this is a list of all paragraph elements of the html
    
    # Save the actual text of the paper as a plain text file
    filename = datapath / (ID + ".txt")
    with open(filename, "w") as f:
        for p in ps:
            t = p.text
            t = t.replace(" Return to the Text", "")
            t = t.replace("Ã\x95", "")
            f.write(t + "\n")
    time.sleep(30)

1


SSLError: HTTPSConnectionPool(host='avalon.law.yale.edu', port=443): Max retries exceeded with url: /18th_century/fed01.asp (Caused by SSLError("Can't connect to HTTPS URL because the SSL module is not available."))

In [8]:
author = []
r = requests.get("https://www.congress.gov/resources/display/content/The+Federalist+Papers")
s = BeautifulSoup(r.text)
table = s.find("table", {"class": "confluenceTable"})
rows = table.findAll("tr")
for row in rows[1:]:
    author.append(row.findAll("td")[2].text)

In [ ]:
### Part 2 - Mimicking the three Federalist authors ###


import nltk
import numpy

nltk.download('punkt')

def draw_word(distrn):
    words = list(distrn)
    freqs = [freq for w, freq in distrn.items()]
    total = sum(freqs)
    probs = [freq/total for freq in freqs]
    return numpy.random.choice(words, p=probs)

def generate_with_trigrams(text, word=None, num=100):
    tokens = nltk.tokenize.word_tokenize(text)
    trigrams = nltk.trigrams(tokens)
    condition_pairs = (((w0, w1), w2) for w0, w1, w2 in trigrams)
    cfdist = nltk.ConditionalFreqDist(condition_pairs)
    if word is None:
        prev = draw_word(nltk.FreqDist(tokens))
        word = draw_word(nltk.ConditionalFreqDist(nltk.bigrams(tokens))[prev])
    elif len(word.split()) == 1:
        prev = word
        word = draw_word(nltk.ConditionalFreqDist(nltk.bigrams(tokens))[prev])
        # will give an error if this pair doesn't show up in the text
    else:
        prev, word = word.split()[:2]
    print(prev, end=' ')
    for i in range(1, num):
        print(word, end=' ')
        prev, word = word, draw_word(cfdist[(prev, word)])


# each author will have all his papers merged
hamilton = ""
madison = ""
jay = ""

docnames = [f for f in os.listdir() if f[-4:]==".txt"]
docnames.sort()

N = len(docnames)
for i in range(N):
    with open(docnames[i], 'r') as f:
        if author[i] == "Hamilton":
            hamilton += f.read() + " "
        elif author[i] == "Madison":
            madison += f.read() + " "
        elif author[i] == "Jay":
            jay += f.read() + " "

len(hamilton)
len(madison)
len(jay)


generate_with_trigrams(hamilton, "The")
generate_with_trigrams(madison, "The")
generate_with_trigrams(jay, "The")

In [ ]:
### Part 3 - Creating data frame of token frequencies ###

import pandas as pd
import re

N = len(docnames)
tables = [None]*N
for i in range(N):
    with open(docnames[i], 'r') as f:
        doc = f.read()
        doc = doc.replace("To the People of the State of New York:", "")
        doc = doc.replace("PUBLIUS", "")
        doc = doc.replace("Ã¥", "")
        doc = re.sub("[0-9]+", "", doc)
        doc = doc.lower()
        tokens = nltk.tokenize.word_tokenize(doc)
        tables[i] = nltk.FreqDist(tokens)

df = pd.DataFrame(tables)

# fill in zeros
df.fillna(0, inplace=True)

# divide rows by totals
for i in range(N):
    s = sum(df.iloc[i])
    df.iloc[i] = [n/s for n in df.iloc[i]]

df.iloc[0] # check a row to make sure it worked

# write as csv
df.to_csv("../federalist.csv")












# write authors as well
with open("../authors.csv", "w") as f:
    f.write("author\n")
    for a in author:
        f.write(a + "\n")

# and write tokens
with open("../tokens.csv", "w") as f:
    f.write("token\n")
    for t in list(df):
        f.write('"' + t + '"\n')